In [31]:
import numpy as np
import pandas as pd
import torch
import torchtext
import torch.nn as nn
from torchtext.legacy import data
import random
import spacy
from transformers import BertTokenizer, BertModel



print(torch.__version__)

1.9.0+cpu


In [3]:
train_df = pd.read_csv("train.csv")

In [4]:
train_df[train_df["target"] == 0]["text"].values[np.random.randint(0, 1000)]

'I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????'

In [5]:
train_df[train_df["target"] == 1]["text"].values[np.random.randint(0, 1000)]

'Buildings are on fire and they have time for a business meeting #TheStrain'

In [6]:
SEED = 998

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# TEXT = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm', include_lengths = True)
# LABEL = data.LabelField(dtype = torch.float)

# datafields = [(None, None), (None, None), (None, None), ("text", TEXT), ("target", LABEL)]

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
len(tokenizer.vocab)

30522

In [18]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

In [19]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [20]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [21]:
datafields = [(None, None), (None, None), (None, None), ("text", TEXT), ("target", LABEL)]

TrainSet = data.TabularDataset("train.csv", "CSV", datafields, skip_header = True)

train_data, valid_data = TrainSet.split(random_state = random.seed(SEED))

In [25]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

In [27]:
LABEL.build_vocab(train_data)

In [28]:
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


In [29]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# train_iterator, valid_iterator = data.BucketIterator.splits((train_data, valid_data), batch_size = BATCH_SIZE, device = device, sort=False)
train_iterator, valid_iterator = data.BucketIterator.splits((train_data, valid_data),
                                                            device = device,
                                                            batch_size = BATCH_SIZE, 
                                                            sort_key = lambda x: len(x.text), 
                                                            sort_within_batch = True)

In [32]:
bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output


In [34]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [37]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [36]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [38]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [39]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [40]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.target)
        
        acc = binary_accuracy(predictions, batch.target)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [41]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.target)
            
            acc = binary_accuracy(predictions, batch.target)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [42]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [43]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 4m 50s
	Train Loss: 0.531 | Train Acc: 74.07%
	 Val. Loss: 0.491 |  Val. Acc: 78.78%
Epoch: 02 | Epoch Time: 4m 53s
	Train Loss: 0.425 | Train Acc: 81.76%
	 Val. Loss: 0.426 |  Val. Acc: 80.73%
Epoch: 03 | Epoch Time: 4m 55s
	Train Loss: 0.398 | Train Acc: 83.43%
	 Val. Loss: 0.407 |  Val. Acc: 82.39%
Epoch: 04 | Epoch Time: 5m 1s
	Train Loss: 0.374 | Train Acc: 84.56%
	 Val. Loss: 0.418 |  Val. Acc: 81.92%
Epoch: 05 | Epoch Time: 4m 56s
	Train Loss: 0.346 | Train Acc: 85.37%
	 Val. Loss: 0.422 |  Val. Acc: 82.60%


In [47]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, valid_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.407 | Test Acc: 82.39%


In [48]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens     = tokenizer.tokenize(sentence)
    tokens     = tokens[:max_input_length-2]
    indexed    = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor     = torch.LongTensor(indexed).to(device)
    tensor     = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [83]:
predict_sentiment(model, tokenizer, "Gun fire heared near a school")

0.948711097240448

In [78]:
predict_sentiment(model, tokenizer, "'Yall are missing out on the thing' - @nobody")

0.17911440134048462